Every year, American high school students take SATs, which are standardized tests intended to measure literacy, numeracy, and writing skills. There are three sections - reading, math, and writing, each with a **maximum score of 800 points**. These tests are extremely important for students and colleges, as they play a pivotal role in the admissions process.

Analyzing the performance of schools is important for a variety of stakeholders, including policy and education professionals, researchers, government, and even parents considering which school their children should attend. 

The dataset "schools.csv" will be used for analyzing the results.

In [19]:
import pandas as pd
import sqlite3

df = pd.read_csv('schools.csv')
df.head()

conn = sqlite3.connect(":memory:")
df.columns = [c.replace(" ", "_") for c in df.columns]
df.to_sql("schools", conn, index=False, if_exists="replace")

#Set up to run SQL queries
def q(sql):
    return pd.read_sql_query(sql, conn)

To view the data

In [20]:
q("SELECT * FROM schools")

,school_name,borough,building_code,average_math,average_reading,average_writing,percent_tested
0,"New Explorations into Science, Technology and ...",Manhattan,M022,657,601,601,NaN
1,Essex Street Academy,Manhattan,M445,395,411,387,78.9
2,Lower Manhattan Arts Academy,Manhattan,M445,418,428,415,65.1
3,High School for Dual Language and Asian Studies,Manhattan,M445,613,453,463,95.9
4,Henry Street School for International Studies,Manhattan,M056,410,406,381,59.7
...,...,...,...,...,...,...,...
370,"Queens High School for Information, Research, ...",Queens,Q465,372,362,352,44.6
371,Rockaway Park High School for Environmental Su...,Queens,Q410,357,381,376,38.5
372,Channel View School for Research,Queens,Q410,427,430,423,76.6
373,Rockaway Collegiate High School,Queens,Q410,399,403,405,46.5


Find out which NYC schools have the best math results, achieving at least 80% of the maximum possible score of 800 in math.

In [21]:
q("""SELECT school_name, average_math
FROM schools
WHERE average_math > 800*0.8
ORDER BY average_math DESC""")

,school_name,average_math
0,Stuyvesant High School,754
1,Bronx High School of Science,714
2,Staten Island Technical High School,711
3,Queens High School for the Sciences at York Co...,701
4,"High School for Mathematics, Science, and Engi...",683
5,Brooklyn Technical High School,682
6,Townsend Harris High School,680
7,High School of American Studies at Lehman College,669
8,"New Explorations into Science, Technology and ...",657
9,Eleanor Roosevelt High School,641


Find out which NYC schools have the best reading results, achieving at least 80% of the maximum possible score of 800 in reading.

In [22]:
q("""SELECT school_name, average_reading
FROM schools
WHERE average_reading > 800*0.8
ORDER BY average_reading DESC""")

,school_name,average_reading
0,Stuyvesant High School,697
1,High School of American Studies at Lehman College,672
2,Staten Island Technical High School,660
3,Bronx High School of Science,660
4,Bard High School Early College,641


Find out which NYC schools have the best writing results, achieving at least 80% of the maximum possible score of 800 in writing.

In [23]:
q("""SELECT school_name, average_writing
FROM schools
WHERE average_writing > 800*0.8
ORDER BY average_writing DESC""")

,school_name,average_writing
0,Stuyvesant High School,693
1,High School of American Studies at Lehman College,672
2,Staten Island Technical High School,670
3,Bronx High School of Science,667
4,Townsend Harris High School,661


Add a new column called "total_SAT" by summing the math, reading, and writing scores in the original DataFrame using Python

In [24]:
df["total_SAT"] = df["average_math"] + df["average_reading"] + df["average_writing"]
print(df)
# Update the SQLite database with the new total_SAT column
df.to_sql("schools", conn, index=False, if_exists="replace")

                                           school_name    borough  \
0    New Explorations into Science, Technology and ...  Manhattan   
1                                 Essex Street Academy  Manhattan   
2                         Lower Manhattan Arts Academy  Manhattan   
3      High School for Dual Language and Asian Studies  Manhattan   
4        Henry Street School for International Studies  Manhattan   
..                                                 ...        ...   
370  Queens High School for Information, Research, ...     Queens   
371  Rockaway Park High School for Environmental Su...     Queens   
372                   Channel View School for Research     Queens   
373                    Rockaway Collegiate High School     Queens   
374                                  Scholars' Academy     Queens   

    building_code  average_math  average_reading  average_writing  \
0            M022           657              601              601   
1            M445           395  

375

Identify the top 10 performing schools based on the combined SAT scores

In [25]:
q("""SELECT school_name, total_SAT
FROM schools 
ORDER BY total_SAT DESC
LIMIT 10""")

,school_name,total_SAT
0,Stuyvesant High School,2144
1,Staten Island Technical High School,2041
2,Bronx High School of Science,2041
3,High School of American Studies at Lehman College,2013
4,Townsend Harris High School,1981
5,Queens High School for the Sciences at York Co...,1947
6,Bard High School Early College,1914
7,Brooklyn Technical High School,1896
8,Eleanor Roosevelt High School,1889
9,"High School for Mathematics, Science, and Engi...",1889


Identify the highest-performing school in each borough

In [26]:
q("""SELECT s.borough, s.school_name, s.total_SAT
FROM schools s
WHERE s.total_SAT = (
    SELECT MAX(total_SAT)
    FROM schools
    WHERE borough = s.borough
)""")

,borough,school_name,total_SAT
0,Manhattan,Stuyvesant High School,2144
1,Staten Island,Staten Island Technical High School,2041
2,Bronx,Bronx High School of Science,2041
3,Brooklyn,Brooklyn Technical High School,1896
4,Queens,Townsend Harris High School,1981


Find which borough has the largest standard deviation in the combined SAT scores by running this code:

In [27]:
q("""SELECT 
    borough, 
    COUNT(school_name) AS num_schools, 
    ROUND(AVG(total_SAT), 2) AS average_SAT, 
    ROUND(SQRT(SUM(POWER(total_SAT - (SELECT AVG(total_SAT) FROM schools s2 WHERE s2.borough = s1.borough), 2)) / (COUNT(*) - 1)), 2) AS std_SAT 
FROM schools s1
GROUP BY borough
ORDER BY std_SAT DESC 
LIMIT 1""")

,borough,num_schools,average_SAT,std_SAT
0,Manhattan,89,1340.13,230.29
